In [1]:
local_outgoing_path = '/home/birniece/Documents/Projects/MicroseismicModelling/ShaheenTransfers/OUTGOING/LargeRockBlock'
shaheen2_path = '/scratch/birniece/LargeRockBlock'
shaheen3_path = '/scratch/birniece/LargeRockBlock'

sofimaster = '/scratch/birniece/sofi3d/bin/sofi3D'

In [2]:
import numpy as np
import pandas as pd
import os
import glob
import scipy 
from scipy.interpolate import RegularGridInterpolator

%matplotlib inline
import matplotlib.pyplot as plt

from pyfrac.utils.sofiutils import *

# FILE PATHS

In [3]:
shaheen_cp_loc = '/home/birniece/Documents/Projects/MicroseismicModelling/ShaheenTransfers/OUTGOING/LargeRockBlock'

# 'FAKE' Shaheen Path
shaheen_proj_loc = '/scratch/birniece/LargeRockBlock/'

rec_file = './inputs/RecArray_fullcoverage.dat'
model_dir = './inputs/model/homo40cubed'

print(shaheen_proj_loc)

/scratch/birniece/LargeRockBlock/


### SLURM FILE CREATION FUNC

In [4]:
def slurmfilecreation(slurmfile, gid, sofimaster, jsonloc, joutloc, write=True, tstring='23:00:00'):
    slurmstr='''#!/bin/bash -l
#SBATCH --job-name=LRB_%.2i
#SBATCH --nodes=32
#SBATCH --partition=workq
#SBATCH --time=%s
#SBATCH --error=sofi.%%J.err 
#SBATCH --output=sofi.%%J.out

#OpenMP settings:
export OMP_NUM_THREADS=1

# RUN MODELLING JOB
srun --hint=nomultithread --ntasks=1000 %s %s.json > %s.jout
'''%(gid, tstring, sofimaster, jsonloc, joutloc)
    
    if write:
        text_file = open(slurmfile, "wt")
        text_file.writelines(slurmstr)
        text_file.close()

# MODELLING PARAMS (Indep. of src locs)

### MODEL PARAMETERS - Get these from the script that made the models

In [5]:
fc_kHz = 25  # kHz
fs_kHz = 100

fc = fc_kHz * 1000
fs = fs_kHz * 1000
rec_dt = 1/fs
print(f"Event Central Frequency: {fc},\nSampling Frequency: {fs},\nReceiver dt: {rec_dt}")

# time params (in seconds!)
dt = 1e-7
tdur = 0.0005
tsnap_params = [dt, dt*2000, dt*50]


Event Central Frequency: 25000,
Sampling Frequency: 100000,
Receiver dt: 1e-05


In [6]:
dimlims = 0.40  # 40cm cubed
dx = dy = dz = 0.01  # 5mm
nx = ny = nz = int(dimlims/dx)
n_xzy = (nx, nz, ny)
d_xzy = [dx, dz, dy]


In [7]:
# BOUNDARIES & FREE SURFACE
nbounds = 0
free_surface = False

In [8]:
# SNAP PARAMETERS
snap_start = dt
snap_end = tdur/2
snap_step = 100*dt
tsnap_params = [snap_start, snap_end, snap_step]
snap_sbsmp_xyz = 1

# SEISMOGRAM PARAMETERS
smgrm_ndt = 100

In [9]:
# GET THE DIFFERENT STRINGS - this will be updated in the future
default_strs = get_default_strings(rec_file=rec_file)

# MODELLING
mod_strs = get_subsurfmod_str(n_xzy, 
                              d_xzy,
                              ' ',
                              moddir=model_dir,
                              np_xzy=[10,10,10]
                             )
bndry_str = get_boundary_str(fs=free_surface, 
                             npad=nbounds, 
                             cpml=True)


# WRITE FILES

## Source file 1 - Explosive, heading out one direction but not overlapping

In [10]:
groupname = "SourceArray_explosiveXfrac"
src_file = './inputs/SourceArray_explosiveXfrac.dat'

# Make time string (long as multishot in one rec)
tdur = 0.6
t_str = get_time_str(dt, tdur)

# Make SOFI Source String
src_str = get_source_str(sfile=src_file, multisource=0)

# Make Monitor String
monitor_strs = get_monitor_str(tsnap_params, 
                           smgrm_ndt, 
                           groupname, 
                           sbsmp_xyz=snap_sbsmp_xyz,
                           snap=False
                          )

# Write SOFI FILE
sofi_json_file = os.path.join(shaheen_cp_loc, groupname+'.json')
shaheen_sofi_json_file = os.path.join(shaheen_proj_loc, groupname+'.json')
print(src_file)
print(sofi_json_file)
write_SOFIjsonParams(default_strs, 
                     monitor_strs,  
                     t_str, 
                     mod_strs, 
                     src_str, 
                     bndry_str,
                     sofi_json_file)

# WRITE SLURM FILE
slurmfile = os.path.join(shaheen_cp_loc, 'submit_'+groupname+'.slurm')
slurmfilecreation(slurmfile, 
                  gid=0, 
                  sofimaster=sofimaster, 
                  jsonloc=shaheen_sofi_json_file[:-5], 
                  joutloc=shaheen_sofi_json_file[:-5],
                 tstring='01:00:00')
print(' ')

./inputs/SourceArray_explosiveXfrac.dat
/home/birniece/Documents/Projects/MicroseismicModelling/ShaheenTransfers/OUTGOING/LargeRockBlock/SourceArray_explosiveXfrac.json
 


## Source file 2 - independent shots, full x-y coverage

In [11]:
groupname = "multishotFullXY"
src_file = './inputs/SourceArray_multishotFullXY.dat'

# Make time string (short as per shot)
tdur = 0.0005
t_str = get_time_str(dt, tdur)

# Make SOFI Source String
src_str = get_source_str(sfile=src_file, multisource=1)

# Make Monitor String
monitor_strs = get_monitor_str(tsnap_params, 
                           smgrm_ndt, 
                           groupname, 
                           sbsmp_xyz=snap_sbsmp_xyz,
                           snap=False
                          )

# Write SOFI FILE
sofi_json_file = os.path.join(shaheen_cp_loc, groupname+'.json')
shaheen_sofi_json_file = os.path.join(shaheen_proj_loc, groupname+'.json')
print(src_file)
print(sofi_json_file)
write_SOFIjsonParams(default_strs, 
                     monitor_strs,  
                     t_str, 
                     mod_strs, 
                     src_str, 
                     bndry_str,
                     sofi_json_file)

# WRITE SLURM FILE
slurmfile = os.path.join(shaheen_cp_loc, 'submit_'+groupname+'.slurm')
slurmfilecreation(slurmfile, 
                  gid=0, 
                  sofimaster=sofimaster, 
                  jsonloc=shaheen_sofi_json_file[:-5], 
                  joutloc=shaheen_sofi_json_file[:-5],)
print(' ')

./inputs/SourceArray_multishotFullXY.dat
/home/birniece/Documents/Projects/MicroseismicModelling/ShaheenTransfers/OUTGOING/LargeRockBlock/multishotFullXY.json
 


# MAKE BASH SCRIPT TO SUBMIT ALL SLURM JOBS

In [12]:
slurmfiles = []
for file in glob.glob(os.path.join(shaheen_cp_loc)+"/*.slurm"):
    slurmfiles.append(file.split('/')[-1])


In [13]:
slurmfiles[0]

'submit_multishotFullXY.slurm'

In [14]:
bashsubmitfile = os.path.join(shaheen_cp_loc, 'submit_ALL.sh')

bashsubmitstring = '''#!/bin/bash 

mkdir -p outputs
mkdir -p outputs/log
mkdir -p outputs/snap
mkdir -p outputs/su

'''

for slurmf in slurmfiles:
    bashsubmitstring+='sbatch %s\n'%slurmf 
print(bashsubmitstring)

text_file = open(bashsubmitfile, "wt")
text_file.writelines(bashsubmitstring)
text_file.close()

#!/bin/bash 

mkdir -p outputs
mkdir -p outputs/log
mkdir -p outputs/snap
mkdir -p outputs/su

sbatch submit_multishotFullXY.slurm
sbatch submit_SourceArray_explosiveXfrac.slurm

